<div class="alert alert-block alert-success">
Doing all my imports
</div>

In [65]:
import pandas as pd
import numpy as np

In [66]:
import requests

In [67]:
from sklearn.cluster import KMeans

In [68]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [69]:
#!conda install -c conda-forge geopy
!pip install geopy

In [70]:
from geopy.geocoders import Nominatim

In [71]:
#!conda install -c conda-forge folium=0.5.0 --yes
!pip install folium
import folium # map rendering library

<div class="alert alert-block alert-success">
Get saved Toronto data
</div>

In [72]:
tor_longlat = pd.read_pickle('tor_longlat_data.pkl')

<div class="alert alert-block alert-success">
Create map of Toronto neighborhoods
</div>

In [73]:
ave_long = tor_longlat["Longitude"].mean()
ave_lat = tor_longlat["Latitude"].mean()

<div class="alert alert-block alert-success">
Map of neigborhoods
</div>

In [74]:
map_toronto = folium.Map(location=[ave_lat, ave_long], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(tor_longlat['Latitude'], tor_longlat['Longitude'], tor_longlat['Borough'], tor_longlat['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<div class="alert alert-block alert-success">
Foursquare credentials
</div>

In [75]:
CLIENT_ID = 'BOIJJKYCZMDIUBCUSMDOXANZCVNFNKWTNS3KJZXEB4PLHQHI' # your Foursquare ID
CLIENT_SECRET = '5EJRIL4HGEV0XWVIORAKGQV225IKNKGLULRAKZGPVHB4RXS0' # your Foursquare Secret
VERSION = '20210715' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BOIJJKYCZMDIUBCUSMDOXANZCVNFNKWTNS3KJZXEB4PLHQHI
CLIENT_SECRET:5EJRIL4HGEV0XWVIORAKGQV225IKNKGLULRAKZGPVHB4RXS0


In [76]:
# What is the maximum radius before I get the same place in different neighborhoods?
#def getPopularPlacesNightsWeekends(names, latitudes, longitudes, radius=500)

In [77]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['id'], # GM - I need this Id so I can get details (premium call)
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue_Id',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [78]:
# This is my dataframe for gathering detailed venue data
venue_details = pd.DataFrame(columns = ["Venue_Id",
                        "Name",
                        "CheckinsCount",
                        "UsersCount",
                        "TipsCount",
                        "VisitsCount",
                        "LikesCount",
                        "Rating",
                        "PriceTier"])

In [93]:
def getVenueDetails(venue_id):
    # NOTE: This is a premium call
    #"price": {
    #    "tier": 3
    #    "message": "Expensive"
    #    "currency": "$"
    #    }
    #"stats": {
    #    "checkinsCount": 364591,
    #    "usersCount": 311634,
    #    "tipCount": 1583,
    #    "visitsCount": 854553
    #  },
    #  "url": "http://www.centralparknyc.org",
    #  "likes": {
    #    "count": 17370,
    #    "summary": "17370 Likes" }
    #"rating": 9.8,
     # create the API request URL
    
    # Don't run this again if we already have the data? 
    
    url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            venue_id,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION
            )
    
    # make the GET request
    result = requests.get(url).json()["response"]['venue']
    #df.loc[len(df.index)] = [value1, value2, value3, ...]
    
    stats = result["stats"]
    pricetier = np.nan
    checkinsCount = np.nan
    usersCount = np.nan
    tipCount = np.nan
    visitsCount = np.nan
    
    if "price" in result:
        price = result["price"]
        if "tier" in price:
            pricetier = price["tier"]
    
    if "checkinsCount" in stats:
        checkinsCount = stats["checkinsCount"]
    if "usersCount" in stats:
        usersCount = stats["usersCount"]
    if "tipCount" in stats:
        tipCount = stats["tipCount"]
    else:
        tipCount = 0
    if "visitsCount" in stats:
        visitsCount = stats["visitsCount"]

        
    venue_details.loc[len(venue_details.index)] = [result["id"],
                                                  result["name"],
                                                  checkinsCount,
                                                  usersCount,
                                                  tipCount,
                                                  visitsCount,
                                                  result["likes"]["count"],
                                                  result["rating"],
                                                  pricetier]
    
    #print(result)
    
                                                   


In [100]:
getVenueDetails('4ad69511f964a520e40721e3')
venue_details.head(10)

,Venue_Id,Name,CheckinsCount,UsersCount,TipsCount,VisitsCount,LikesCount,Rating,PriceTier
0,5c08474447f876002c736b68,WVRST,NaN,NaN,8,NaN,20,8.2,NaN
1,4ae8c1d8f964a52005b221e3,The National Club,NaN,NaN,5,NaN,17,7.3,2.0
2,4ad4c05df964a5205df620e3,Pravda Vodka Bar,NaN,NaN,56,NaN,104,7.9,3.0
3,53361f24498e6a4d4d8f66f1,D.W. Alexander,NaN,NaN,71,NaN,103,8.3,3.0
4,50ecb1fae4b0beb13294f0aa,John & Sons Oyster House,NaN,NaN,5,NaN,21,8.3,3.0
5,4ad69511f964a520e40721e3,The Keg Steakhouse + Bar - York Street,NaN,NaN,79,NaN,269,8.8,2.0


In [123]:
venue_details.to_pickle("venue_details.pkl")

In [98]:
# I want to see some really popular places to test the details results
toronto_popular = toronto_venues[(toronto_venues["Neighborhood"] == "Commerce Court, Victoria Hotel") & (toronto_venues["Venue Category"].str.contains("Restaurant"))]
toronto_popular.head(30)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue_Id,Venue,Venue Latitude,Venue Longitude,Venue Category
893,"Commerce Court, Victoria Hotel",43.648198,-79.379817,4ad4c05df964a52059f620e3,Canoe,43.647452,-79.381320,Restaurant
901,"Commerce Court, Victoria Hotel",43.648198,-79.379817,55fc571f498ec35023360858,Cactus Club Cafe,43.649552,-79.381671,American Restaurant
902,"Commerce Court, Victoria Hotel",43.648198,-79.379817,4ae7199ff964a52068a821e3,Bymark,43.647217,-79.381252,Restaurant
904,"Commerce Court, Victoria Hotel",43.648198,-79.379817,4ae73054f964a5203ca921e3,Ki Modern Japanese + Bar,43.647223,-79.379374,Japanese Restaurant
907,"Commerce Court, Victoria Hotel",43.648198,-79.379817,5a4fdf56772fbc5e9fa73c7f,Chotto Matte,43.646473,-79.378782,Japanese Restaurant
913,"Commerce Court, Victoria Hotel",43.648198,-79.379817,50e44770e4b0e03a48c0d8a8,Kupfert & Kim (First Canadian Place),43.648547,-79.381624,Gluten-free Restaurant
917,"Commerce Court, Victoria Hotel",43.648198,-79.379817,506db1a9e4b0a3f3b31412f0,Richmond Station,43.651569,-79.379266,American Restaurant
918,"Commerce Court, Victoria Hotel",43.648198,-79.379817,50ecb1fae4b0beb13294f0aa,John & Sons Oyster House,43.650656,-79.381613,Seafood Restaurant
921,"Commerce Court, Victoria Hotel",43.648198,-79.379817,5214e7c111d2a83379eae21f,The Chase,43.650952,-79.379422,New American Restaurant
927,"Commerce Court, Victoria Hotel",43.648198,-79.379817,4ad69511f964a520e40721e3,The Keg Steakhouse + Bar - York Street,43.649987,-79.384103,Restaurant


In [63]:
getVenueDetails('4ae6ea6ef964a52082a721e3')
venue_details.head()

,Venue_Id,Name,CheckinsCount,UsersCount,TipsCount,VisitsCount,LikesCount,Rating
0,4b168a88f964a52028ba23e3,lululemon,0,0,5,0,12,7.2
1,4aec6bbaf964a520e6c621e3,Starbucks,0,0,9,0,36,6.9
2,4ae6ea6ef964a52082a721e3,Jules Cafe Patisserie,0,0,12,0,22,8.9


In [83]:
names = tor_longlat["Neighborhood"]
longitude = tor_longlat["Longitude"]
latitude = tor_longlat["Latitude"]
toronto_venues = getNearbyVenues(names, latitude, longitude)
toronto_venues.head(50)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue_Id,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,50e6da19e4b0d8a78a0e9794,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,5082ef77e4b0a7491cf7b022,Zodiac Swim School,43.728532,-79.382860,Swim School
2,Lawrence Park,43.728020,-79.388790,50ed9da8e4b081eabee12672,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
3,Davisville North,43.712751,-79.390197,4ba011c2f964a5204a5737e3,Sherwood Park,43.716551,-79.387776,Park
4,Davisville North,43.712751,-79.390197,4e8e73c30cd6209590ae7be4,Summerhill Market North,43.715499,-79.392881,Food & Drink Shop
5,Davisville North,43.712751,-79.390197,4adb2fd3f964a520c42421e3,Homeway Restaurant & Brunch,43.712641,-79.391557,Breakfast Spot
6,Davisville North,43.712751,-79.390197,560f20eb498ef7d04038f8cd,Winners,43.713236,-79.393873,Department Store
7,Davisville North,43.712751,-79.390197,4b7810c3f964a52030b42ee3,Best Western Roehampton Hotel & Suites,43.708878,-79.390880,Hotel
8,Davisville North,43.712751,-79.390197,4b0b3691f964a520c62e23e3,Subway,43.708474,-79.390674,Sandwich Place
9,Davisville North,43.712751,-79.390197,4c3f2724db3b1b8d635e6695,900 Mount Pleasant - Residents Gym,43.711671,-79.391767,Gym / Fitness Center


<H1>Save Toronto Venues Data</H1>

In [101]:
toronto_venues.to_pickle("toronto_venues_data.pkl")

<div class="alert alert-block alert-success">
I believe there were <b>way too many venue categories</b> in the FourSquare data, so I manually created a much simplified set of 10 categories which I want to cluster on.
</div>

In [151]:
simple_categories = pd.read_csv('FourSquare Fun Categories - V5.csv')
simple_categories.head()

,Entertainment,Recreation,CheapMeal,CoffeeDessert,Ethnic Food,Fancy Food,Bar,FunStore,NotFunCheapMeal,Store,FoodStore,Travel,Misc
0,Theater,Park,Café,Coffee Shop,Pizza Place,Seafood Restaurant,Bar,Bookstore,Fast Food Restaurant,Clothing Store,Fish Market,Hotel,Bank
1,Concert Hall,Gym,Restaurant,Bakery,Italian Restaurant,Steakhouse,Pub,Art Gallery,Airport Food Court,Pharmacy,Cheese Shop,Rental Car Location,Intersection
2,Movie Theater,Gym / Fitness Center,Sandwich Place,Dessert Shop,Japanese Restaurant,French Restaurant,Gastropub,Antique Shop,College Cafeteria,Sporting Goods Shop,Grocery Store,Light Rail Station,Salon / Barbershop
3,Aquarium,Yoga Studio,Breakfast Spot,Tea Room,Sushi Restaurant,New American Restaurant,Liquor Store,Flea Market,NaN,Department Store,Beer Store,Train Station,Plaza
4,Jazz Club,Spa,Burger Joint,Juice Bar,Thai Restaurant,Bistro,Beer Bar,Candy Store,NaN,Convenience Store,Supermarket,Airport Service,Office


In [130]:
fun_places = ['Entertainment','Recreation','CheapMeal','CoffeeDessert','Ethnic Food','Fancy Food','Bar','FunStore']

def get_simple_category(venue_type):
    #print(venue_type)
    venue_type = str.strip(venue_type)
    # It is WRONG to use contains here - type = "hotel" matches "hotel bar" and "hotel pool" which are in different categories
    # column_name = [col for col in simple_categories.columns if simple_categories[col].str.contains(venue_type).any()]
    column_name = [col for col in simple_categories.columns if (simple_categories[col] == venue_type).any()]
    if len(column_name) == 0:
        result="NOT FOUND"
        print("NOT FOUND: " + venue_type)
    elif column_name[0] in fun_places:
        result = column_name[0]
    else:
        result="NOT FUN!"
    return result
    

<div class="alert alert-block alert-success">
Assign the simple categories to the main venues dataframe
</div>

In [143]:
#Dataframe.apply(customFunction, axis=0)
#df['A'] = df['A'].apply(add_2)
toronto_venues["SimpleCategory"] = toronto_venues["Venue Category"].apply(get_simple_category)
toronto_venues.head()

NOT FOUND: Food


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue_Id,Venue,Venue Latitude,Venue Longitude,Venue Category,SimpleCategory
0,Lawrence Park,43.728020,-79.388790,50e6da19e4b0d8a78a0e9794,Lawrence Park Ravine,43.726963,-79.394382,Park,Recreation
1,Lawrence Park,43.728020,-79.388790,5082ef77e4b0a7491cf7b022,Zodiac Swim School,43.728532,-79.382860,Swim School,Recreation
2,Lawrence Park,43.728020,-79.388790,50ed9da8e4b081eabee12672,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line,NOT FUN!
3,Davisville North,43.712751,-79.390197,4ba011c2f964a5204a5737e3,Sherwood Park,43.716551,-79.387776,Park,Recreation
4,Davisville North,43.712751,-79.390197,4e8e73c30cd6209590ae7be4,Summerhill Market North,43.715499,-79.392881,Food & Drink Shop,NOT FUN!
5,Davisville North,43.712751,-79.390197,4adb2fd3f964a520c42421e3,Homeway Restaurant & Brunch,43.712641,-79.391557,Breakfast Spot,CheapMeal
6,Davisville North,43.712751,-79.390197,560f20eb498ef7d04038f8cd,Winners,43.713236,-79.393873,Department Store,NOT FUN!
7,Davisville North,43.712751,-79.390197,4b7810c3f964a52030b42ee3,Best Western Roehampton Hotel & Suites,43.708878,-79.390880,Hotel,NOT FUN!
8,Davisville North,43.712751,-79.390197,4b0b3691f964a520c62e23e3,Subway,43.708474,-79.390674,Sandwich Place,CheapMeal
9,Davisville North,43.712751,-79.390197,4c3f2724db3b1b8d635e6695,900 Mount Pleasant - Residents Gym,43.711671,-79.391767,Gym / Fitness Center,Recreation


In [146]:
toronto_venues[toronto_venues["Venue Category"] == "Hotel"][10:30]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue_Id,Venue,Venue Latitude,Venue Longitude,Venue Category,SimpleCategory
670,"Richmond, Adelaide, King",43.650571,-79.384568,597ae286b2958f7e52761fe8,The Adelaide Hotel Toronto,43.649831,-79.380164,Hotel,NOT FUN!
674,"Richmond, Adelaide, King",43.650571,-79.384568,4b0563c0f964a5200e5822e3,Marriott Downtown at CF Toronto Eaton Centre,43.654728,-79.382422,Hotel,NOT FUN!
702,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,53357710498e20817350cfb4,Delta Hotels by Marriott Toronto,43.642882,-79.383949,Hotel,NOT FUN!
711,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,4cb87a334c60a093c04a39ca,Le Germain Hotel,43.643125,-79.380918,Hotel,NOT FUN!
763,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,534af3d411d24fecc553d2f3,Radisson Admiral Hotel Toronto-Harbourfront,43.638765,-79.385871,Hotel,NOT FUN!
768,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,4ad4c05bf964a5209bf520e3,"The Westin Harbour Castle, Toronto",43.641211,-79.375749,Hotel,NOT FUN!
795,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,4ad4c05bf964a520a7f520e3,The Fairmont Royal York,43.645449,-79.381508,Hotel,NOT FUN!
830,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,4e31b74252b131dcebb08743,Shangri-La Toronto,43.649129,-79.386557,Hotel,NOT FUN!
837,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,4d2ce8d1fd2e236a8ac8ebde,The Ritz-Carlton,43.645330,-79.387089,Hotel,NOT FUN!
851,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,4af96fbbf964a520c01122e3,One King West Hotel & Residence,43.649139,-79.377876,Hotel,NOT FUN!


In [105]:
toronto_venues.to_pickle("toronto_venues_fun.pkl")

In [132]:
# Remove all the venues that are not fun
toronto_venues_fun = toronto_venues[toronto_venues["SimpleCategory"] != "NOT FUN!"]
toronto_venues_fun.to_pickle("toronto_venues_fun.pkl")

<ipython-input-109-e2f0ce4e322b>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  toronto_venues_fun[toronto_venues["SimpleCategory"] == "NOT FUN!"].head()


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue_Id,Venue,Venue Latitude,Venue Longitude,Venue Category,SimpleCategory


In [133]:
# one hot encoding
# toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot = pd.get_dummies(toronto_venues_fun[['SimpleCategory']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues_fun['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Bar,CheapMeal,CoffeeDessert,Entertainment,Ethnic Food,Fancy Food,FunStore,NOT FOUND,Recreation
0,Lawrence Park,0,0,0,0,0,0,0,0,1
1,Lawrence Park,0,0,0,0,0,0,0,0,1
3,Davisville North,0,0,0,0,0,0,0,0,1
5,Davisville North,0,1,0,0,0,0,0,0,0
8,Davisville North,0,1,0,0,0,0,0,0,0


In [19]:
#toronto_onehot = toronto_onehot.drop(['NOT FOUND','NOT FUN!'],axis=1)
toronto_onehot = toronto_onehot.drop(['NOT FUN!'],axis=1)

In [135]:
toronto_onehot = toronto_onehot.drop(['NOT FOUND'],axis=1)

In [136]:
toronto_onehot[30:60]

,Neighborhood,Bar,CheapMeal,CoffeeDessert,Entertainment,Ethnic Food,Fancy Food,FunStore,Recreation
42,Davisville,0,0,0,0,1,0,0,0
43,Davisville,0,0,1,0,0,0,0,0
44,Davisville,0,0,0,0,1,0,0,0
45,Davisville,0,1,0,0,0,0,0,0
46,Davisville,1,0,0,0,0,0,0,0
47,Davisville,0,0,0,0,0,0,0,1
48,Davisville,0,0,0,0,0,0,0,1
49,Davisville,0,0,0,0,1,0,0,0
50,Davisville,0,0,0,0,1,0,0,0
51,Davisville,0,1,0,0,0,0,0,0


In [137]:
toronto_sum = toronto_onehot.groupby('Neighborhood').sum().reset_index()
#df.sum(axis=1)
toronto_sum["TotalFun"] = toronto_sum.sum(axis=1)
#df.sort_values(by=['c1'])
toronto_sum = toronto_sum.sort_values(by=["TotalFun"],ascending=False).reset_index()
toronto_sum.head(30)

,index,Neighborhood,Bar,CheapMeal,CoffeeDessert,Entertainment,Ethnic Food,Fancy Food,FunStore,Recreation,TotalFun
0,33,"First Canadian Place, Underground city",9,18,14,5,26,7,2,6,87
1,18,"Commerce Court, Victoria Hotel",8,22,17,2,20,6,3,9,87
2,39,"Harbourfront East, Union Station, Toronto Islands",10,12,20,15,12,4,1,11,85
3,86,"Toronto Dominion Centre, Design Exchange",10,16,16,6,21,7,2,3,81
4,78,"Stn A PO Boxes, 25 The Esplanade, Enclave of M5E",11,11,15,5,18,7,5,7,79
5,67,"Richmond, Adelaide, King",8,13,12,5,26,4,3,6,77
6,35,"Garden District, Ryerson",4,6,16,4,21,3,5,8,67
7,15,Church and Wellesley,9,7,9,2,25,1,1,10,64
8,75,St. James Town,10,10,8,1,20,6,2,5,62
9,13,Central Bay Street,2,13,19,1,17,3,1,5,61


In [138]:
toronto_sum.to_pickle("toronto_most_fun.pkl")

<h1>----------End of Initial Toronto Fun Data----------</h1>

<h1>--------Getting New York Data--------</h1>

In [113]:
ny_long_lat = pd.read_pickle("nyc_long_lat.pkl")

In [23]:
ny_long_lat.head(30)

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
5,Bronx,Kingsbridge,40.881687,-73.902818
6,Manhattan,Marble Hill,40.876551,-73.910660
7,Bronx,Woodlawn,40.898273,-73.867315
8,Bronx,Norwood,40.877224,-73.879391
9,Bronx,Williamsbridge,40.881039,-73.857446


In [114]:
names = ny_long_lat["Neighborhood"]
longitude = ny_long_lat["Longitude"]
latitude = ny_long_lat["Latitude"]
nyc_venues = getNearbyVenues(names, latitude, longitude)
nyc_venues.head(20)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue_Id,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,4c537892fd2ea593cb077a28,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,4d6af9426107f04dedeb297a,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,4c783cef3badb1f7e4244b54,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,5d5f5044d0ae1c0008f043c3,Walgreens,40.896528,-73.844700,Pharmacy
4,Wakefield,40.894705,-73.847201,4c25c212f1272d7f836385c5,Dunkin',40.890459,-73.849089,Donut Shop
5,Wakefield,40.894705,-73.847201,508af256e4b0578944c87392,Cooler Runnings Jamaican Restaurant,40.898083,-73.850259,Caribbean Restaurant
6,Wakefield,40.894705,-73.847201,4d33665fb6093704b80001e0,Subway,40.890468,-73.849152,Sandwich Place
7,Wakefield,40.894705,-73.847201,4e440828a809d4ed1bb7059f,Pitman Deli,40.896744,-73.844398,Food
8,Wakefield,40.894705,-73.847201,4f32458019836c91c7c734ff,Central Deli,40.896728,-73.844387,Deli / Bodega
9,Wakefield,40.894705,-73.847201,5681717c498e9b9cf4d8c187,Koss Quick Wash,40.891281,-73.849904,Laundromat


In [147]:
nyc_venues["SimpleCategory"] = nyc_venues["Venue Category"].apply(get_simple_category)
nyc_venues.head(30)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue_Id,Venue,Venue Latitude,Venue Longitude,Venue Category,SimpleCategory
0,Wakefield,40.894705,-73.847201,4c537892fd2ea593cb077a28,Lollipops Gelato,40.894123,-73.845892,Dessert Shop,CoffeeDessert
2,Wakefield,40.894705,-73.847201,4c783cef3badb1f7e4244b54,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop,CoffeeDessert
4,Wakefield,40.894705,-73.847201,4c25c212f1272d7f836385c5,Dunkin',40.890459,-73.849089,Donut Shop,CoffeeDessert
5,Wakefield,40.894705,-73.847201,508af256e4b0578944c87392,Cooler Runnings Jamaican Restaurant,40.898083,-73.850259,Caribbean Restaurant,Ethnic Food
6,Wakefield,40.894705,-73.847201,4d33665fb6093704b80001e0,Subway,40.890468,-73.849152,Sandwich Place,CheapMeal
8,Wakefield,40.894705,-73.847201,4f32458019836c91c7c734ff,Central Deli,40.896728,-73.844387,Deli / Bodega,CheapMeal
11,Co-op City,40.874294,-73.829939,4d2cfa5cad25224bbbc5fb8f,Capri II Pizza,40.876374,-73.829940,Pizza Place,Ethnic Food
14,Co-op City,40.874294,-73.829939,4cb2e08f562d224bf5a62988,Bagels on Bartow,40.870280,-73.828611,Bagel Shop,CheapMeal
17,Co-op City,40.874294,-73.829939,53d55899498e2d92dfb89804,Dunkin',40.869812,-73.829473,Donut Shop,CoffeeDessert
18,Co-op City,40.874294,-73.829939,4be2b79d660ec9284d04ca3b,Townhouse Restaurant,40.876086,-73.828868,Restaurant,CheapMeal


In [154]:
nyc_venues[nyc_venues["Venue Category"].str.contains("Hotel")][0:50]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue_Id,Venue,Venue Latitude,Venue Longitude,Venue Category,SimpleCategory
3825,Upper East Side,40.775639,-73.960508,3fd66200f964a520cdea1ee3,Bemelmans Bar,40.774607,-73.963308,Hotel Bar,Bar
4366,Clinton,40.759101,-73.996119,4f6e221de4b0e0075b3fa222,yotel galley,40.759182,-73.995513,Hotel Bar,Bar
5157,Little Italy,40.719324,-73.997305,5bc7ea96f1936e002c929b24,CloudM Rooftop Bar,40.720615,-73.993537,Hotel Bar,Bar
5246,Soho,40.722184,-74.000657,4acbd878f964a520ffc720e3,The Crosby Bar,40.722980,-73.997324,Hotel Bar,Bar
6265,Flushing,40.764454,-73.831773,52ed58ef498e6f393547100a,Oo Bar & Rooftop Lounge,40.763628,-73.828432,Hotel Bar,Bar
6398,East Elmhurst,40.764073,-73.867041,4db352730437fa536a126006,Elements Lounge,40.767433,-73.867048,Hotel Bar,Bar
6615,College Point,40.784903,-73.843045,50e25be4e4b0a78b0afcf73c,View De Point,40.781346,-73.841053,Hotel Bar,Bar
7525,North Corona,40.754071,-73.857518,4b5498eef964a520d3c127e3,The Pine Restaurant,40.754661,-73.853925,Hotel Bar,Bar
8458,Noho,40.723259,-73.988434,5642aef9498e51025cf4a7a5,Mr. Purple,40.721949,-73.987952,Hotel Bar,Bar
8580,Civic Center,40.715229,-74.005415,545a8c16498eeafe6d52b176,Evening Bar,40.715301,-74.009347,Hotel Bar,Bar


In [116]:
nyc_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue_Id,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,4c537892fd2ea593cb077a28,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,4d6af9426107f04dedeb297a,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,4c783cef3badb1f7e4244b54,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,5d5f5044d0ae1c0008f043c3,Walgreens,40.896528,-73.844700,Pharmacy
4,Wakefield,40.894705,-73.847201,4c25c212f1272d7f836385c5,Dunkin',40.890459,-73.849089,Donut Shop


In [155]:
# Remove all not found and not fun - not found categories are all not fun - I checked
nyc_venues = nyc_venues[ (nyc_venues['SimpleCategory']!='NOT FOUND') | (nyc_venues['SimpleCategory']!='NOT FUN!') ]

In [156]:
# Save all the nyc venues data which is fun
nyc_venues.to_pickle("nyc venues only fun.pkl")

In [31]:
# get names of indexes for which
# column Age has value 21
#index_names = nyc_venues[ nyc_venues['SimpleCategory'] == 'NOT FOUND' ].index
  
# drop these row indexes
# from dataFrame
#nyc_venues.drop(index_names, inplace = True)

In [142]:
# one hot encoding
nyc_onehot = pd.get_dummies(nyc_venues[['SimpleCategory']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
nyc_onehot['Neighborhood'] = nyc_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [nyc_onehot.columns[-1]] + list(nyc_onehot.columns[:-1])
nyc_onehot = nyc_onehot[fixed_columns]

nyc_onehot.head()
#nyc_onehot = nyc_onehot.drop(['NOT FUN!'],axis=1)
nyc_sum = nyc_onehot.groupby('Neighborhood').sum().reset_index()
nyc_sum["TotalFun"] = nyc_sum.sum(axis=1)
nyc_sum = nyc_sum.sort_values(by=["TotalFun"],ascending=False).reset_index()

# Save all the fun data for nyc
nyc_sum.to_pickle("nyc_most_fun.pkl")

nyc_sum.head(30)

,index,Neighborhood,Bar,CheapMeal,CoffeeDessert,Entertainment,Ethnic Food,Fancy Food,FunStore,Recreation,TotalFun
0,184,Murray Hill,11,19,14,2,59,5,0,13,123
1,252,South Side,22,10,9,2,35,3,1,8,90
2,6,Astoria,14,13,13,0,38,4,0,7,89
3,81,East Village,20,7,11,2,40,1,5,3,89
4,194,North Side,17,7,21,2,29,2,2,6,86
5,298,Yorkville,9,13,12,1,30,3,1,17,86
6,285,West Village,15,4,10,6,29,8,2,12,86
7,276,Upper West Side,13,11,12,3,36,4,2,4,85
8,94,Financial District,8,11,16,4,22,6,0,16,83
9,73,Downtown,7,13,11,4,28,4,5,11,83


<h1>Fun Popular Focus Idea</h1>
<p>
I'm thinking I will take the top 20 neighborhoods based on the Fun Total for both Toronto and NYC
Then I will take all the fun places in those neighborhoods and get data for number of reviews and total score.
Then I will come up with a formula which assigns a fun weighting to each simple category and then does a calculation involving the number of reviews and the total score - and then I will aggregate these back together and get a total fun result.
</p>

In [30]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head(30)

,Neighborhood,Bar,CheapMeal,CoffeeDessert,Entertainment,Ethnic Food,Fancy Food,FunStore,Recreation
0,Agincourt,0.250000,0.250000,0.000000,0.000000,0.250000,0.000000,0.000000,0.250000
1,"Alderwood, Long Branch",0.125000,0.125000,0.125000,0.000000,0.250000,0.000000,0.000000,0.250000
2,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.181818,0.136364,0.000000,0.227273,0.000000,0.045455,0.000000
3,Bayview Village,0.000000,0.250000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000
4,"Bedford Park, Lawrence Manor East",0.083333,0.166667,0.166667,0.000000,0.375000,0.000000,0.000000,0.000000
5,Berczy Park,0.140351,0.087719,0.140351,0.070175,0.192982,0.087719,0.035088,0.052632
6,"Birch Cliff, Cliffside West",0.000000,0.250000,0.000000,0.500000,0.000000,0.000000,0.000000,0.250000
7,"Brockton, Parkdale Village, Exhibition Place",0.040000,0.240000,0.200000,0.120000,0.080000,0.000000,0.000000,0.120000
8,"Business reply mail Processing Centre, 969 Eas...",0.062500,0.062500,0.000000,0.062500,0.125000,0.000000,0.062500,0.250000
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.187500,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.062500


In [152]:


num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    # T - Transpose index and columns. Reflect the DataFrame over its main diagonal by writing rows as columns and vice-versa. The property T is an accessor to the method
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
           venue  freq
0            Bar  0.33
1      CheapMeal  0.33
2    Ethnic Food  0.33
3  CoffeeDessert  0.00
4  Entertainment  0.00


----Alderwood, Long Branch----
           venue  freq
0     Recreation  0.33
1    Ethnic Food  0.22
2            Bar  0.11
3      CheapMeal  0.11
4  CoffeeDessert  0.11


----Bathurst Manor, Wilson Heights, Downsview North----
           venue  freq
0      CheapMeal  0.21
1    Ethnic Food  0.21
2          Store  0.21
3  CoffeeDessert  0.12
4           Misc  0.12


----Bayview Village----
           venue  freq
0    Ethnic Food  0.50
1      CheapMeal  0.25
2           Misc  0.25
3            Bar  0.00
4  CoffeeDessert  0.00


----Bedford Park, Lawrence Manor East----
           venue  freq
0    Ethnic Food  0.36
1      CheapMeal  0.24
2  CoffeeDessert  0.12
3          Store  0.12
4            Bar  0.08


----Berczy Park----
         venue  freq
0          Bar  0.17
1  Ethnic Food  0.17
2    FoodStore  0.12
3        Store  0.12
4   

4  CoffeeDessert   0.1


----North Park, Maple Leaf Park, Upwood Park----
           venue  freq
0     Recreation  0.50
1  CoffeeDessert  0.25
2           Misc  0.25
3            Bar  0.00
4      CheapMeal  0.00


----North Toronto West----
           venue  freq
0          Store  0.25
1    Ethnic Food  0.20
2     Recreation  0.20
3      CheapMeal  0.15
4  CoffeeDessert  0.10


----Northwest, Clairville, Humberwood, Woodbine Downs, West Humber, Kipling Heights, Rexdale, Elms, Tandridge, Old Rexdale----
           venue  freq
0         Travel  0.50
1  CoffeeDessert  0.25
2          Store  0.25
3            Bar  0.00
4      CheapMeal  0.00


----Northwood Park, York University----
           venue  freq
0  CoffeeDessert  0.25
1    Ethnic Food  0.25
2          Store  0.25
3     Recreation  0.12
4         Travel  0.12


----Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East----
           venue  freq
0     

In [102]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

<div class="alert alert-block alert-success">
<b>THIS WILL LOOK MUCH DIFFERENT BECAUSE I AM USING MY OWN LIST OF SIMPLIFIED VENUE CATEGORIES</b>
</div>

In [153]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(25)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Ethnic Food,CheapMeal,Bar,Travel,Store,Recreation,Misc,FoodStore,Fancy Food,Entertainment
1,"Alderwood, Long Branch",Recreation,Ethnic Food,Store,CoffeeDessert,CheapMeal,Bar,Travel,Misc,FoodStore,Fancy Food
2,"Bathurst Manor, Wilson Heights, Downsview North",Store,Ethnic Food,CheapMeal,Misc,CoffeeDessert,FoodStore,Recreation,Travel,Fancy Food,Entertainment
3,Bayview Village,Ethnic Food,Misc,CheapMeal,Travel,Store,Recreation,FoodStore,Fancy Food,Entertainment,CoffeeDessert
4,"Bedford Park, Lawrence Manor East",Ethnic Food,CheapMeal,Store,CoffeeDessert,FoodStore,Bar,Travel,Recreation,Misc,Fancy Food
5,Berczy Park,Ethnic Food,Bar,Store,FoodStore,CoffeeDessert,CheapMeal,Fancy Food,Entertainment,Recreation,Travel
6,"Birch Cliff, Cliffside West",Entertainment,Recreation,CheapMeal,Travel,Store,Misc,FoodStore,Fancy Food,Ethnic Food,CoffeeDessert
7,"Brockton, Parkdale Village, Exhibition Place",CheapMeal,CoffeeDessert,Store,Recreation,Misc,Ethnic Food,Entertainment,FoodStore,Bar,Travel
8,"Business reply mail Processing Centre, 969 Eas...",Recreation,Travel,Store,Ethnic Food,CheapMeal,Misc,FoodStore,Bar,Fancy Food,Entertainment
9,"CN Tower, King and Spadina, Railway Lands, Har...",Travel,CoffeeDessert,CheapMeal,Bar,Store,Recreation,Misc,FoodStore,Fancy Food,Ethnic Food


In [154]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 4, 1, 3, 3, 1, 1, 1, 4, 1])

In [155]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

neighborhoods_venues_sorted['Cluster Labels'].value_counts()

1    35
3    28
4    18
0    11
2     8
Name: Cluster Labels, dtype: int64

In [167]:
# add clustering labels - already done
# neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = tor_longlat

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4N,Central Toronto,Lawrence Park,M4N,43.728020,-79.388790,4.0,Recreation,Travel,Misc,Store,FoodStore,Fancy Food,Ethnic Food,Entertainment,CoffeeDessert,CheapMeal
1,M4P,Central Toronto,Davisville North,M4P,43.712751,-79.390197,4.0,Recreation,Store,CheapMeal,FoodStore,Bar,Travel,Misc,Fancy Food,Ethnic Food,Entertainment
2,M4R,Central Toronto,North Toronto West,M4R,43.715383,-79.405678,1.0,Store,Recreation,Ethnic Food,CheapMeal,CoffeeDessert,Travel,Misc,FoodStore,Fancy Food,Entertainment
3,M4S,Central Toronto,Davisville,M4S,43.704324,-79.388790,3.0,Ethnic Food,CheapMeal,CoffeeDessert,Recreation,Store,FoodStore,Misc,Fancy Food,Bar,Travel
4,M4T,Central Toronto,"Moore Park, Summerhill East",M4T,43.689574,-79.383160,4.0,Recreation,Misc,Travel,Store,FoodStore,Fancy Food,Ethnic Food,Entertainment,CoffeeDessert,CheapMeal


<div class="alert alert-block alert-success">
Try to come up with ideas for what my clusters represent actually.
</div>

In [195]:
 #,toronto_merged['Cluster Labels'] == 3]].value_counts() # 0 - Store is first, lots of cheap food and recreation
#test_df = toronto_merged[toronto_merged['Cluster Values','1st Most Common Venue','2nd Most Common Venue','3rd Most Common Venue']]
test_df = toronto_merged[['Cluster Labels','1st Most Common Venue','2nd Most Common Venue','3rd Most Common Venue']]
test_df[test_df['Cluster Labels'] == 4].head(60)
#0 - LOTS OF STORES AND RECREATION

#.groupby(['Cluster Labels']).agg(lambda x:x.value_counts())
#test_df.groupby(['Cluster Labels']).size().unstack(fill_value=0)   
# 0 - Stores, Recreation, Food
# 1 - Cheap food is first, ethnic restaurant 2nd,
# 2 - Recreation, Travel, Stores
# 3 - Ethnic Food, Coffee, Desert
# 4 - Recreation, Travel, Miscellaneous (like 2)

,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,4,Recreation,Travel,Misc
1,4,Recreation,Store,CheapMeal
4,4,Recreation,Misc,Travel
6,4,Recreation,Misc,CheapMeal
7,4,Recreation,Store,Ethnic Food
27,4,Recreation,Misc,FoodStore
31,4,Recreation,Travel,Store
38,4,Recreation,Ethnic Food,Store
40,4,Recreation,Misc,Travel
50,4,Recreation,Ethnic Food,CheapMeal


In [188]:
# toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged[toronto_merged['Cluster Labels'].isnull()]
toronto_merged.drop(index=[42,53,91],inplace=True)

In [189]:
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)

<div class="alert alert-block alert-success">
<b>NOTE TO PEER REVIEWER: THIS WILL LOOK MUCH DIFFERENT BECAUSE I AM USING MY OWN LIST OF SIMPLIFIED VENUE CATEGORIES. I BELIEVE THIS IS A SUBSTANTIAL IMPROVEMENT FOR CLUSTERING.</b>
</div>

In [196]:
# create map
map_clusters = folium.Map(location=[ave_lat, ave_long], zoom_start=11)

# Cluster Names
cluster_types = ['Stores, Recreation, Food','Cheap food, ethnic restaurants','Recreation, Travel, Stores','Ethnic Food, Coffee, Dessert Places', 'Recreation, Travel, Miscellaneous']

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + cluster_types[cluster], parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters